# Week 4: Advanced RasterIO

Individual learning outcomes: At the end of this week, all students should be able to call RasterIO commands from within Python to reproject or reshape a Sentinel-2 image.

# Connect to Google Drive from Colab

In [ ]:
# Load the Drive helper and mount your Google Drive as a drive in the virtual machine
from google.colab import drive
drive.mount('/content/drive')

# Import libraries

In [ ]:
# install some libraries that are not on Colab by default
!pip install rasterio
!pip install geopandas
!pip install rasterstats
!pip install sentinelsat

#import required libraries
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import plot
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import numpy as np
import os
from os.path import join
from pprint import pprint
import sys

# import the pygge library of functions for this module
# make sure that this path points to the location of the pygge module on your Google Drive
libdir = '/content/drive/MyDrive/practicals21-22' # this is where pygge.py needs to be saved
if libdir not in sys.path:
    sys.path.append(libdir)
# import the pygge module
import pygge

# define the root directory where our data are stored - adjust this to your directory names
rootdir = '/content/drive/MyDrive/practicals21-22/'

%matplotlib inline

Make sure before running the next cell, you have uploaded the Sentinel-2 image to your Google Drive. Check in the directory structure on the left hand side of Colab whether it is there. You may need to adapt the file path and file names if you are using a different Sentinel-2 image.

# Open image files

Last week, we learned how to create Geotiff files with 3 bands for visualisation purposes and how to show them as true colour and false colour composites. We worked with the four bands that are available at 10 m spatial resolution. However, Sentinel-2 also has some other really good bands at 20 m and even 60 m resolution: https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/resolutions/radiometric

Today, we want to work with more image bands. We will create a Geotiff image in uint16 data format (remember what that is from last week?). It will contain all 10m and 20 m resolution bands. We will resample the four 10m bands to 20m resolution to make them compatible.

In [ ]:
#open bands as separate single-band raster from the image directory pointing to the 20 m resolution bands
imagePath = join(rootdir, 'S2A_MSIL2A_20180507T110621_N0207_R137_T30UXD_20180507T131836.SAFE/GRANULE/L2A_T30UXD_A015006_20180507T110835/IMG_DATA/R20m/')
#note: explore the directory and file structure in the data explorer on the left hand side of Colab (click on the folder icon)
band2 = rasterio.open(imagePath+'T30UXD_20180507T110621_B02_20m.jp2', driver='JP2OpenJPEG') #blue
band3 = rasterio.open(imagePath+'T30UXD_20180507T110621_B03_20m.jp2', driver='JP2OpenJPEG') #green
band4 = rasterio.open(imagePath+'T30UXD_20180507T110621_B04_20m.jp2', driver='JP2OpenJPEG') #red
band5 = rasterio.open(imagePath+'T30UXD_20180507T110621_B05_20m.jp2', driver='JP2OpenJPEG') #vegetation red edge 1
band6 = rasterio.open(imagePath+'T30UXD_20180507T110621_B06_20m.jp2', driver='JP2OpenJPEG') #vegetation red edge 2
band7 = rasterio.open(imagePath+'T30UXD_20180507T110621_B07_20m.jp2', driver='JP2OpenJPEG') #vegetation red edge 3
band8A = rasterio.open(imagePath+'T30UXD_20180507T110621_B8A_20m.jp2', driver='JP2OpenJPEG') #narrow-band nir
bandAOT = rasterio.open(imagePath+'T30UXD_20180507T110621_AOT_20m.jp2', driver='JP2OpenJPEG') #aerosol optical thickness
bandSCL = rasterio.open(imagePath+'T30UXD_20180507T110621_SCL_20m.jp2', driver='JP2OpenJPEG') #automatic scene classification
bandTCI = rasterio.open(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', driver='JP2OpenJPEG') #True Colour Image in uint8 data format
bandWVP = rasterio.open(imagePath+'T30UXD_20180507T110621_WVP_20m.jp2', driver='JP2OpenJPEG') #Water vapour pressure

# True Colour Image

There are a lot more image bands than we used last week. Let's explore some of them.
We start with the TCI 'band'. We shall see that in fact the TCI is a jpeg2000 file with 3 bands included in the same file - a red, green and blue channel. But the TCI image is a quicklook image, i.e. its resolution has been downsampled to make the file smaller and quicker to visualise.

In [ ]:
#looking at the data types of this array, we can see that the three bands are all uint8 data format
print(bandTCI.dtypes)

In [ ]:
# create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(21,7))
fig.patch.set_facecolor('white')

#plot band using RasterIO
plot.show(bandTCI, ax=ax1)

# zoom in to an area of interest by setting limits for each 'axis', meaning subplot
ax2.set_xlim([640000,650000])
ax2.set_ylim([5840000,5850000])
plot.show(bandTCI, ax=ax2)

# zoom in further and to a different place
ax3.set_xlim([622421, 630421])
ax3.set_ylim([5829707, 5837707])
plot.show(bandTCI, ax=ax3)

Alternatively, we can use the easy_plot function from the pygge library. It is slower but more versatile and automatically visualises any data type, not just uint8 data.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(21,7)) # create a figure with 3 subplots
fig.patch.set_facecolor('white')

pygge.easy_plot(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', ax1, bands=[1,2,3], xlim=None, ylim=None)
pygge.easy_plot(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', ax2, bands=[1,2,3], xlim=[640000,650000], ylim=[5840000,5850000])
pygge.easy_plot(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', ax3, bands=[1,2,3], xlim=[622421, 630421], ylim=[5829707, 5837707])

easy_plot also allows you to choose a single band and visualise it with a colour map. We try this now with the same band from the TCI image file (which has 3 bands in total). Each map will use a different colour map.


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(21,7)) # create a figure with 3 subplots
fig.patch.set_facecolor('white')

pygge.easy_plot(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', ax1, bands=[1], xlim=[640000,650000], ylim=[5840000,5850000])
pygge.easy_plot(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', ax2, bands=[1], cmap="rainbow", xlim=[640000,650000], ylim=[5840000,5850000])
pygge.easy_plot(imagePath+'T30UXD_20180507T110621_TCI_20m.jp2', ax3, bands=[1], cmap="Spectral", xlim=[640000,650000], ylim=[5840000,5850000])

Last week, we had to apply various contrast stretches to improve the visual appearance of the image. The TCI band offers a quick and ready-made way of visualising the true colour composite without having to do that. It uses the threshold of 2000 to exclude any extreme values from the bands, as we did last week. That is why we used this value.

# Water Vapour Image

Let's look at the water vapour band. This is being used to atmospherically correct the land surface reflectance image and shows us the contamination of the top-of-atmosphere radiance image (L1C, which we have not used) with a water vapour signal. Water vapour tends to be highest in the humid tropics.

In [ ]:
#this array only has one band, so we visualise it with a single-band colour scale
#we also have to rescale it to uint8 format for visualisation, as last week
print(bandWVP.dtypes)

In [ ]:
#plot the water vapour band
WVP = bandWVP.read(1)
img = pygge.scale_to_uint8(WVP)
print("WVP band min = ", WVP.min())
print("WVP band max = ", WVP.max())
print("rescaled min = ", img.min())
print("rescaled max = ", img.max())
plot.show(img, cmap='Blues')

# Aerosol Optical Thickness

Moving on to aerosol optical thickness, which is another atmospheric effect that the ESA L2A processing chain corrects for to get from top-of-atmosphere radiance to land surface reflectance.

In [ ]:
#this array only has one band, so we visualise it with a single-band colour scale
#we also have to rescale it to uint8 format for visualisation, as last week
print(bandAOT.dtypes)

Like water vapour, this file has only one band with data type uint16.

In [ ]:
#plot the aerosol optical thickness band
AOT = bandAOT.read(1)
img = pygge.scale_to_uint8(AOT)
print("AOT band min = ", AOT.min())
print("AOT band max = ", AOT.max())
print("rescaled min = ", img.min())
print("rescaled max = ", img.max())
plot.show(img, cmap='Greys')

# Automatic Scene Classification Image

Now onto the scene classification file. ESA applies an automatic scene classification to the Sentinel-2 images to get from L1C to L2A, because its atmospheric correction algorithm needs some information on the land cover type.
This is described here.

In [ ]:
#this array only has one band in uint8 format
#pixel values indicate classes
print(bandSCL.dtypes)

In [ ]:
# read in the band data
#SCL = bandSCL.read(1);
# import colour map library
from matplotlib import cm
from matplotlib.colors import ListedColormap

# make a new colour map
viridis = cm.get_cmap('viridis', 12)
SCLcmap = ListedColormap(viridis(np.linspace(0, 1, 12)))

# assign colours to the SCL classes 
# the values mean red, green, blue intensity, and transparency (alpha) and must be between 0 (low) and 1 (high)
SCLcmap.colors[0] = [0., 0., 0., 1.] # No Data = black
SCLcmap.colors[1] = [1., 0., 0.016, 1.] # Saturated / Defective = red
SCLcmap.colors[2] = [0.525, 0.525, 0.525, 1.] # Dark Area Pixels = grey
SCLcmap.colors[3] = [0.467, 0.298, 0.043, 1.] # Cloud Shadows = brown
SCLcmap.colors[4] = [0.063, 0.827, 0.176, 1.] # Vegetation = green
SCLcmap.colors[5] = [1., 1., 0.325, 1.] # Bare Soils = yellow
SCLcmap.colors[6] = [0., 0., 1., 1.] # Water = blue
SCLcmap.colors[7] = [0.506, 0.506, 0.506, 1.] # Clouds low probability / Unclassified = medium grey
SCLcmap.colors[8] = [0.753, 0.753, 0.753, 1.] # Clouds medium probability = light grey
SCLcmap.colors[9] = [0.949, 0.949, 0.949, 1.] # Clouds high probability = very light grey
SCLcmap.colors[10] = [0.733, 0.773, 0.925, 1.] # Cirrus clouds = light blue/purple
SCLcmap.colors[11] = [0.325, 1., 0.980, 1.] # Snow / Ice = cyan

# print our colour map 
print('SCLcmap.colors', SCLcmap.colors)

'''
modified from: https://www.sentinel-hub.com/faq/how-get-s2a-scene-classification-sentinel-2/
'''          

Now we can plot our scene classification map with our own colour table.

In [ ]:
# create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(21,7))
fig.patch.set_facecolor('white')

#plot band using RasterIO - Remember we did this last week
plot.show(bandSCL, ax=ax1, cmap=SCLcmap)

# zoom in to an area of interest by setting limits for each 'axis', meaning subplot
ax2.set_xlim([640000,650000])
ax2.set_ylim([5840000,5850000])
plot.show(bandSCL, ax=ax2, cmap=SCLcmap)

# zoom in further and to a different place
ax3.set_xlim([622421, 630421])
ax3.set_ylim([5829707, 5837707])
plot.show(bandSCL, ax=ax3, cmap=SCLcmap)

#Navigating the directory structure on your Google Drive from a Python program
To see a list of all images and scripts in your current directory, type:

In [ ]:
print("Going to content directory.")
os.chdir("/content")
print("We are currently in this directory: ", os.getcwd())
print("It has these contents:")
!ls -l
print("")

# go to a different directory by using the os library
os.chdir("drive")
print("Now we are in this directory: ", os.getcwd())
print("It has these contents:")
!ls -l
print("")

# go to a different directory
os.chdir('MyDrive')
print("Now we are in this directory: ", os.getcwd())
print("It has these contents:")
!ls -l
print("")

# go to our root directory for this practical
os.chdir(rootdir)
print("Now we are in this directory: ", os.getcwd())
print("It has these contents:")
print(os.listdir())
print("")
!ls -l


# Band stacking

Stack all bands we want to retain in one GeoTiff file.

In [ ]:
# save all image bands as a Geotiff file
s2stack = rasterio.open(join(rootdir, 'Sentinel-2_stack.tiff'), 'w', driver='Gtiff', 
                             width=band4.width, height=band4.height, count=7, 
                             crs=band4.crs, transform=band4.transform, 
                             dtype=np.uint16)
s2stack.write(band2.read(1), 1)
s2stack.write(band3.read(1), 2)
s2stack.write(band4.read(1), 3)
s2stack.write(band5.read(1), 4)
s2stack.write(band6.read(1), 5)
s2stack.write(band7.read(1), 6)
s2stack.write(band8A.read(1), 7)
s2stack.close()

In [ ]:
s2stack = rasterio.open(join(rootdir, 'Sentinel-2_stack.tiff'))

# print some metadata of the band 4 file before we stacked the bands
print("Single band file before stacking:")
print("number of raster bands:", band4.count)
print("number of raster columns:", band4.width)
print("number of raster rows:", band4.height)
print("")

# print some metadata of the new stacked file
print("Stacked Geotiff file:")
print("number of raster bands:", s2stack.count)
print("number of raster columns:", s2stack.width)
print("number of raster rows:", s2stack.height)
print("")

# look at the Geotiff file properties and file size
os.chdir(rootdir)
print("Now we are in this directory: ", os.getcwd())
print("This is the file we created:")
!ls -l *_stack.tiff
print("")

# get the filename of our new file using the os library
allfiles = os.listdir()
matches = [f for f in allfiles if "_stack.tiff" in f]
print(matches)

#Image Resampling and Reprojection

In Python we can resample an image to a different spatial resolution and reproject it onto a different map projection.
 If we want to create a quicklook image at 100 m resolution, we can use RasterIO. It is based on the GDAL library.

Now let's run this command on our stacked Sentinel-2 image. We will create a new raster file called s2stacked_100m.tif with 100 m resolution and the same projection as our original image.

We are going to produce a new image called 'Sentinel-2_stack_100m_v2.tif' using GDAL from within Python rather than from the command line.

In [ ]:
# get EPSG code of the coordinate referencing system of the raster file
#   because we do not want to change it
with rasterio.open('Sentinel-2_stack.tiff') as src:
  epsg = src.crs["init"].split(":")[-1]
  print("EPSG code of the file ", "Sentinel-2_stack.tiff", epsg)
  src.close()

# run the resampling algorithm
pygge.easy_warp('Sentinel-2_stack.tiff',
                'Sentinel-2_stack_100m.tiff', 
                epsg, 
                res=Resampling.bilinear,
                xres=100, 
                yres=100)

Let's look at the number of bands, pixels and lines in the different image files we have created.

In [ ]:
s2stack100m = rasterio.open(join(rootdir, 'Sentinel-2_stack_100m.tiff'))

# print some metadata of the first file
print("Stacked Geotiff file:")
print("number of raster bands:", s2stack100m.count)
print("number of raster columns:", s2stack100m.width)
print("number of raster rows:", s2stack100m.height)
print("")

s2stack100m.close()

Visualise the 20 m and the two 100 m resolution images on screen.

In [ ]:
# create a figure with 2x2 subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(21,21))
fig.patch.set_facecolor('white')

#plot the images with full extent
pygge.easy_plot(join(rootdir, 'Sentinel-2_stack.tiff'), ax=ax1, bands=[1,2,3], percentiles=[0,99.5])
pygge.easy_plot(join(rootdir, 'Sentinel-2_stack_100m.tiff'), ax=ax2, bands=[1,2,3], percentiles=[0,99.5])

# zoom in to an area of interest
xlim=[640000,650000]
ylim=[5840000,5850000]
pygge.easy_plot(join(rootdir, 'Sentinel-2_stack.tiff'), ax=ax3, bands=[1,2,3], percentiles=[0,99.5], xlim=xlim, ylim=ylim)
pygge.easy_plot(join(rootdir, 'Sentinel-2_stack_100m.tiff'), ax=ax4, bands=[1,2,3], percentiles=[0,99.5], xlim=xlim, ylim=ylim)


Note that there are other resampling options available. We chose averaging adjacent pixels to the coarser resolution. Alternatives include nearest neighbour resampling and bilinear interpolation. Try them out in the GDAL Translate command and create another image file. Look at the difference that makes.

Let's check the projection of this new image using rasterio.

In [ ]:
src = rasterio.open('Sentinel-2_stack_100m.tiff')
# convert the coordinate referencing system information to well-known text (wkt)
d = src.crs.to_wkt() 
print(type(d))
pprint(d)
src.close()

#Reprojecting an image
Sometimes we want to change the geographic projection of an image, for example to make it compatible with national map projection requirements, or if we want to mosaic different Sentinel-2 images together to a larger map. Their projection changes depending on the longitude.

This is also called image warping. Imagine you are streching a rubber sheet over an orange. This is essentially what you do here. The orange is your ellipsoid model of the shape of the Earth.

For example, we can change the image from UTM projection to the British National Grid projection: https://epsg.io/27700


In [ ]:
dst_epsg = 27700 # destination EPSG code

# call the easy_warp function from the pygge module
pygge.easy_warp('Sentinel-2_stack_100m.tiff', 'Sentinel-2_stack_100m_BNG.tiff', dst_epsg)

# print the metadata of the warped file
src = rasterio.open('/content/drive/MyDrive/practicals21-22/Sentinel-2_stack_100m_BNG.tiff')
d = src.crs.to_wkt() 
print(type(d))
pprint(d)
src.close()

You can see from the metadata that the new image file is now in British National Grid projection.

Let's print the new BNG image next to the UTM image to see how much difference this makes.

We have to be mindful that the coordinate system in BNG and UTM projection has different scales. This means that we have to convert our coordinates for the zoom window from one projection to the other before calling our plotting function.

In [ ]:
s2stack100mUTMfile = '/content/drive/MyDrive/practicals21-22/Sentinel-2_stack_100m.tiff'
s2stack100mBNGfile = '/content/drive/MyDrive/practicals21-22/Sentinel-2_stack_100m_BNG.tiff'

s2stack100mUTM = rasterio.open(s2stack100mUTMfile)
s2stack100mBNG = rasterio.open(s2stack100mBNGfile)

# create a figure with 2x2 subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(17,17))
fig.patch.set_facecolor('white')

#plot the images with full extent
pygge.easy_plot(s2stack100mUTMfile, ax=ax1, percentiles=[0,99.5])
pygge.easy_plot(s2stack100mBNGfile, ax=ax2, percentiles=[0,99.5])

# zoom in to an area of interest in UTM coordinates
xlim=[640000,650000]
ylim=[5840000,5850000]

# convert from UTM to BNG coordinates (using numpy arrays)
xs = np.array(xlim)
ys = np.array(ylim)
print("Coordinates in UTM projection: ", xlim, ylim)

# first convert UTM coordinates to pixel locations in the image
pxl = rasterio.transform.rowcol(s2stack100mUTM.transform, xs, ys)
print("Coordinates in pixels and rows: ", pxl)

# then convert the pixel locations to BNG coordinates
bng = rasterio.transform.xy(s2stack100mBNG.transform, pxl[0], pxl[1])
print("Coordinates in BNG projection: ", bng)

#plot the zoom images
pygge.easy_plot(s2stack100mUTMfile, ax=ax3, percentiles=[0,99.5], xlim=xlim, ylim=ylim)
pygge.easy_plot(s2stack100mBNGfile, ax=ax4, percentiles=[0,99.5], xlim=bng[0], ylim=bng[1])

You can probably see small black margins around the overview image in BNG project. This is caused by the warping and shows that it worked.

# NDVI


The Normalized Difference Vegetation Index (NDVI) is an indicator of the proportion and condition of green vegetation. Generally for surfaces with some vegetation the value of NDVI is positive, for surfaces without vegetation the value is null, while for water and clouds the value is usually negative. The closer to the positive end, the higher the density of the vegetation cover, that is, it is consistent with its dense and developed stage. This value gradually decreases for less dense vegetation cover, which has positive but not very high values.

In [ ]:
red = s2stack100mBNG.read(3) # band 3 in our stacked image
nir = s2stack100mBNG.read(7) # band 7 in our stacked image

# NDVI calculation as flaoting point array
ndvi = (nir.astype(float) - red.astype(float)) / (nir.astype(float) + red.astype(float))

# Ignore division by zero
np.seterr(divide='ignore', invalid='ignore')

# print some image statistics, ignoring missing values (nan)
print("minimum NDVI = ", np.nanmin(ndvi))
print("mean NDVI = ", np.nanmean(ndvi))
print("maximum NDVI = ", np.nanmax(ndvi))
print("standard deviation = ", np.nanstd(ndvi))

# save the NDVI image as a Geotiff file
ndvifile = rasterio.open('/content/drive/MyDrive/practicals20-21/Sentinel-2_NDVI_100m.tiff',
                          'w',driver='Gtiff', width=s2stack100mBNG.width, height=s2stack100mBNG.height,
                          count=1, crs=s2stack100mBNG.crs, transform=s2stack100mBNG.transform, 
                          dtype=np.float64)
ndvifile.write(ndvi, 1)
ndvifile.close()

Because we have some missing values (nan) in the NDVI image due to pixels for which NDVI could not be computed, we want to improve our function to make a uint8 image for visualisation such that it excludes nan values. If we do not do that, it will return an image with only nan values for all pixels. We also want to exclude any pixels with a negative NDVI from the visualisation.

In [ ]:
# save the NDVI image as a scaled uint8 Geotiff file for plotting
img = pygge.scale_to_uint8(ndvi) # OLD: np.where(ndvi>0, ndvi, 0)
ndvifile = rasterio.open('/content/drive/MyDrive/practicals20-21/Sentinel-2_NDVI_100m_uint8.tiff',
                          'w',driver='Gtiff', width=s2stack100mBNG.width, height=s2stack100mBNG.height,
                          count=1, crs=s2stack100mBNG.crs, transform=s2stack100mBNG.transform, 
                          dtype=np.uint8)
ndvifile.write(img, 1)
ndvifile.close()

imgfile = rasterio.open('/content/drive/MyDrive/practicals20-21/Sentinel-2_NDVI_100m_uint8.tiff', 'r')

# create a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
fig.patch.set_facecolor('white')
plot.show(imgfile, ax=ax1, cmap='Greens')

# zoom in to an area of interest
ax2.set_xlim(bng[0])
ax2.set_ylim(bng[1])
plot.show(imgfile, ax=ax2, cmap='Greens')

# Formative assignment for this week

Write a cell of code that calculates the Normalised Difference Water Index. It uses bands 8 and 12 of Sentinel-2. The formula can be found at https://foodsecurity-tep.net/S2_NDWI#:~:text=The%20NDWI%20results%20from%20the,water%20content%20of%20the%20vegetation.
Plot a map showing the results.
